<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/_homework_1_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [1]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [2]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.4626564746359752e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [4]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

3.000216200157016e-12


array([-4.15324419e-08, -9.09068218e-09,  5.30635819e-09,  2.80009499e-08,
       -1.54238360e-08,  3.04259453e-08, -1.55500767e-10, -2.61292121e-08,
       -3.56399865e-08,  1.10783861e-08,  3.08036333e-08, -1.05724849e-08,
        5.82770437e+01, -3.28664271e-08,  4.47430701e-08,  8.87564872e-09,
       -1.88909737e-08,  6.11377681e-09, -1.93927162e-08, -1.53831133e-08,
        4.12841217e-08, -8.06351149e-08,  2.01400956e-08, -1.79042304e-08,
       -7.00038370e-08, -4.03560586e-08,  2.08298571e+01,  2.14498640e+01,
       -2.75712732e-08, -1.40943983e-08, -3.47014403e-09, -1.44634023e-08,
        3.12031381e-08, -2.97275900e-08, -2.29478670e-10,  6.67998340e+01,
        1.57908891e-08, -6.43582969e-09, -4.88958735e-09,  8.69434787e-09,
       -3.68757788e-08,  8.43913493e+01, -3.95354475e-08, -4.10191241e-08,
       -2.00425053e-08, -1.75068143e-08,  8.32485251e-09,  3.32581688e-09,
        5.65101053e+01, -1.16540987e-08,  1.34720595e-08,  1.95497962e-08,
       -2.00608271e-08, -

***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс.

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и второй варианты) или l2 (третий и четвертый варианты) регуляризацию.

In [5]:
class LinearRegression:
    def __init__(self, alpha=0.0001, l_ratio=0.001, tol=0.001, max_iter=1000):
        """
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации (в данном примере не используется)
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        """
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
        self.weights = None

    def fit(self, X, y):
        """
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        """
        # Добавляем столбец единиц для учета свободного члена (intercept)
        X = np.c_[np.ones(X.shape[0]), X]
        self.weights = np.zeros(X.shape[1])

        for iteration in range(self.max_iter):
            predictions = np.dot(X, self.weights)

            gradient = np.dot(X.T, (predictions - y)) / len(y)

            self.weights -= self.alpha * gradient

            if iteration > 0:
                if np.linalg.norm(self.weights - prev_weights) < self.tol:
                    break

            prev_weights = np.copy(self.weights)

    def predict(self, X):
        """
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        """
        # Добавляем столбец единиц для учета свободного члена (intercept)
        X = np.c_[np.ones(X.shape[0]), X]
        return np.dot(X, self.weights)

In [6]:
my_reg = LinearRegression(max_iter=1000, alpha=0.1)
my_reg.fit(X, y)
print(mean_squared_error(y, my_reg.predict(X)))
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

9.434655038024892e-05
You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso, **первый и второй вариант**) или с l2-регуляризацией (from sklearn.linear_model import Ridge, **третий и четвертый вариант**)
* со значением параметра регуляризации **0.1 - для первого и третьего варианта, 0.01 - для второго и четвертого варианта**.

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [10]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


# Создание и обучение модели Ridge с L2-регуляризацией
ridge_model = Ridge(alpha=0.1)
ridge_model.fit(X, y)
ridge_mse = mean_squared_error(y, ridge_model.predict(X))
print(f'MSE (Ridge): {ridge_mse}')

my_reg = LinearRegression(alpha=0.1)
my_reg.fit(X, y)

custom_mse = mean_squared_error(y, my_reg.predict(X))
print(f'MSE (Custom): {custom_mse}')

MSE (Ridge): 2.797983347520257e-06
MSE (Custom): 9.434655038024892e-05


***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.